In [1]:
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture(0)

In [3]:
detector = cv2.CascadeClassifier("../haarcascade_frontalface_default.xml")

In [4]:
flag= True

while(flag):
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(frame,1.3,5)
        if(len(faces)):
            x,y,w,h = faces[0]
            flag=False
            track_window = (x, y, w, h)

            # set up the ROI for tracking
            roi = frame[y:y+h, x:x+w]
            hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
            cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

            # Setup the termination criteria, either 10 iteration or move by at least 1 pt
            term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

            while(True):
                ret, frame = cap.read()

                if ret == True:
                    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                    dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

                    # apply camshift to get the new location
                    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

                    # Draw it on image
                    #print(ret,track_window)
                    pts = cv2.boxPoints(ret)
                    #print("points",pts)
                    pts = np.int0(pts)
                    #print("after np int",pts)
                    img2 = cv2.polylines(frame,[pts],True, 255,2)
#                     img = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
                    cv2.imshow('Frame',img2)

                    k = cv2.waitKey(30) & 0xff
                    if k == 27:
                        break
                else:
                    break
cap.release()
cv2.destroyAllWindows()

((333.0, 301.5), (205.08273315429688, 213.6378173828125), 57.415863037109375) (242, 214, 182, 175)
points [[187.77187 272.62448]
 [367.7834  157.57251]
 [478.22815 330.37552]
 [298.2166  445.4275 ]]
after np int [[187 272]
 [367 157]
 [478 330]
 [298 445]]
((325.0, 302.0), (192.7154541015625, 203.3258056640625), 56.94228744506836) (239, 220, 172, 164)
points [[187.23257 276.69595]
 [357.64432 165.78506]
 [462.76743 327.30405]
 [292.35568 438.21494]]
after np int [[187 276]
 [357 165]
 [462 327]
 [292 438]]
((328.0, 304.0), (191.79281616210938, 195.76454162597656), 113.22318267822266) (237, 215, 182, 178)
points [[275.86182 177.27719]
 [455.7647  254.46982]
 [380.13818 430.7228 ]
 [200.23529 353.53018]]
after np int [[275 177]
 [455 254]
 [380 430]
 [200 353]]
((323.0, 305.5), (191.9799041748047, 199.179443359375), 12.351888656616211) (228, 207, 190, 197)
points [[207.92825 382.25073]
 [250.53574 187.68185]
 [438.07175 228.74927]
 [395.46426 423.31815]]
after np int [[207 382]
 [250 187

((379.0, 240.5), (471.3373107910156, 568.3611450195312), 88.8960189819336) (118, 4, 522, 473)
points [[ 9.0331558e+01  1.0350372e+01]
 [ 6.5858722e+02 -6.0018921e-01]
 [ 6.6766846e+02  4.7064963e+02]
 [ 9.9412781e+01  4.8160019e+02]]
after np int [[ 90  10]
 [658   0]
 [667 470]
 [ 99 481]]
((378.0, 242.5), (467.7692565917969, 575.35791015625), 86.05481719970703) (116, 8, 524, 469)
points [[ 74.91101   28.96251 ]
 [648.9055   -10.623291]
 [681.089    456.03748 ]
 [107.09448  495.6233  ]]
after np int [[ 74  28]
 [648 -10]
 [681 456]
 [107 495]]
((382.0, 244.0), (476.56927490234375, 569.957275390625), 89.8062744140625) (124, 8, 516, 472)
points [[ 96.21731     6.6802673]
 [666.1714      4.753174 ]
 [667.7827    481.31973  ]
 [ 97.82861   483.24683  ]]
after np int [[ 96   6]
 [666   4]
 [667 481]
 [ 97 483]]
((381.0, 239.5), (471.82757568359375, 564.3724975585938), 87.13351440429688) (122, 3, 518, 473)
points [[ 87.36909   17.99318 ]
 [651.0354   -10.230392]
 [674.6309   461.00684 ]
 [1

 [ 90.88501  458.36243 ]]
after np int [[122 -15]
 [675  21]
 [643 495]
 [ 90 458]]
((383.0, 239.5), (479.8287048339844, 562.3753051757812), 83.2647933959961) (126, 0, 514, 479)
points [[ 75.61548   34.219315]
 [634.10974  -31.736618]
 [690.3845   444.7807  ]
 [131.89026  510.73663 ]]
after np int [[ 75  34]
 [634 -31]
 [690 444]
 [131 510]]
((385.0, 240.0), (479.4128723144531, 558.6529541015625), 91.39862060546875) (130, 0, 510, 480)
points [[111.60753    -6.4528503]
 [670.09406     7.1828003]
 [658.39246   486.45285  ]
 [ 99.905945  472.8172   ]]
after np int [[111  -6]
 [670   7]
 [658 486]
 [ 99 472]]
((386.0, 242.0), (479.2738037109375, 554.489990234375), 89.61580657958984) (132, 4, 508, 476)
points [[1.0715437e+02 4.2275238e+00]
 [6.6163190e+02 5.0944519e-01]
 [6.6484564e+02 4.7977246e+02]
 [1.1036810e+02 4.8349054e+02]]
after np int [[107   4]
 [661   0]
 [664 479]
 [110 483]]
((385.0, 238.0), (474.14300537109375, 546.85400390625), 87.22038269042969) (130, 0, 510, 476)
points [[

((383.0, 240.0), (483.11370849609375, 557.8258666992188), 88.48535919189453) (126, 0, 514, 480)
points [[ 97.79958    5.899872]
 [655.43054   -8.844788]
 [668.20044  474.10013 ]
 [110.56946  488.8448  ]]
after np int [[ 97   5]
 [655  -8]
 [668 474]
 [110 488]]
((384.0, 241.0), (482.4295654296875, 547.3148193359375), 88.88383483886719) (128, 2, 512, 478)
points [[105.695755    5.1616974]
 [652.90674    -5.4997406]
 [662.30426   476.83832  ]
 [115.09326   487.49976  ]]
after np int [[105   5]
 [652  -5]
 [662 476]
 [115 487]]
((387.0, 238.0), (474.029541015625, 552.3798217773438), 89.27033996582031) (134, 0, 506, 476)
points [[107.81419     4.5216217]
 [660.1492     -2.512741 ]
 [666.1858    471.4784   ]
 [113.85083   478.51276  ]]
after np int [[107   4]
 [660  -2]
 [666 471]
 [113 478]]
((387.0, 243.0), (479.4179382324219, 544.07421875), 89.8829116821289) (134, 6, 506, 474)
points [[114.47361    3.847458]
 [658.54663    2.735611]
 [659.52637  482.15253 ]
 [115.45337  483.2644  ]]
afte

In [5]:
type(ret)

tuple